In [83]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

In [18]:
pd.set_option('display.max_columns', 500)

# Business

In [86]:
bz = pd.read_json('yelp_dataset/business.json', lines=True)
# bz = json.load('yelp_dataset/business.json')
# with open('yelp_dataset/business.json') as f:
#     bz = json.load(f)

JSONDecodeError: Extra data: line 2 column 1 (char 340)

In [6]:
bz.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,{'GoodForKids': 'False'},"Golf, Active Life",None
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-..."
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,33.455613,-112.395596,5.0,3,1,None,"Insurance, Financial Services","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,4,1,"{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...","Plumbing, Shopping, Local Services, Home Servi...","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ..."


In [9]:
bz.shape

(192609, 14)

In [13]:
bz.drop(['address','postal_code','latitude','longitude','hours'],axis=1,inplace=True)

In [14]:
check = bz.drop(['attributes'],axis=1)
pd.DataFrame({'count':check.count(),'unique':check.nunique(),'null count':check.isnull().sum()})

,count,unique,null count
business_id,192609,192609,0
name,192609,145046,0
city,192609,1204,0
state,192609,36,0
stars,192609,9,0
review_count,192609,1184,0
is_open,192609,2,0
categories,192127,93385,482


In [23]:
bz.head()

,business_id,name,city,state,stars,review_count,is_open,attributes,categories
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,Phoenix,AZ,3.0,5,0,{'GoodForKids': 'False'},"Golf, Active Life"
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,Mississauga,ON,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,Charlotte,NC,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese"
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,Goodyear,AZ,5.0,3,1,None,"Insurance, Financial Services"
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,Charlotte,NC,4.0,4,1,"{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...","Plumbing, Shopping, Local Services, Home Servi..."


In [68]:
def unpack(df, column, fillna=None):
    ret = None
    if fillna is None:
        tmp = pd.DataFrame((d for idx, d in df[column].iteritems()))
        ret = pd.concat([df.drop(column,axis=1), tmp], axis=1)
    else:
        tmp = pd.DataFrame((d for idx, d in 
        df[column].iteritems())).fillna(fillna)
        ret = pd.concat([df.drop(column,axis=1), tmp], axis=1)
    return ret

In [72]:
x = bz['attributes'][:5].apply(pd.Series)
x

,GoodForKids,RestaurantsReservations,GoodForMeal,BusinessParking,Caters,NoiseLevel,RestaurantsTableService,RestaurantsTakeOut,RestaurantsPriceRange2,OutdoorSeating,BikeParking,Ambience,HasTV,WiFi,Alcohol,RestaurantsAttire,RestaurantsGoodForGroups,RestaurantsDelivery,BusinessAcceptsCreditCards,BusinessAcceptsBitcoin,ByAppointmentOnly
0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,True,True,"{'dessert': False, 'latenight': False, 'lunch'...","{'garage': False, 'street': False, 'validated'...",True,u'loud',True,True,2,False,False,"{'romantic': False, 'intimate': False, 'classy...",False,u'no',u'full_bar',u'casual',True,False,NaN,NaN,NaN
2,True,True,"{'dessert': False, 'latenight': False, 'lunch'...","{'garage': False, 'street': False, 'validated'...",False,u'average',True,True,2,False,True,"{'romantic': False, 'intimate': False, 'touris...",True,u'no',u'beer_and_wine','casual',True,False,True,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,True


In [82]:
import ast
x['GoodForMeal'].where(x['GoodForMeal'].isnull(),ast.literal_eval(x['GoodForMeal']))

ValueError: malformed node or string: 0                                                  NaN
1    {'dessert': False, 'latenight': False, 'lunch'...
2    {'dessert': False, 'latenight': False, 'lunch'...
3                                                  NaN
4                                                  NaN
Name: GoodForMeal, dtype: object

# User

In [15]:
us = pd.read_csv('sample_dataset/user_sample.csv')

In [19]:
us.head()

,Unnamed: 0,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,888070,TQnJM4pv9pxwRt64Gj5JfA,Amy,5,2012-11-04 11:58:43,5,0,1,NaN,"Sxnf9uOrAIy7HOM3NYyF-g, dLC-DsQGkmWF--7076CWmA",0,5.00,0,0,0,0,0,0,0,0,0,1,0
1,786620,_I148UPrAgo-SZ9qOkwX3A,Pat,1,2018-10-24 20:15:13,0,0,0,NaN,"WitMA5a9h_1XQAsb34Xtmw, GUsHfw4QmOaVmXvoqWDD1A...",0,4.00,0,0,0,0,0,0,0,0,0,0,0
2,43384,1e4CMKgujUHagjQgv3NVCA,Carlos,11,2014-06-14 18:07:23,1,2,2,NaN,"WQQsl6_9QhqgJY0Zr5gMnQ, wX2ETi3RJMb6pRmN7Y3jSA...",0,4.50,0,0,0,0,0,0,0,0,0,0,0
3,12854,F0nR2cEBLSZtBb2hlxnpoA,May,159,2006-06-22 04:29:33,173,59,74,"2015,2016,2017","InVoTvQufmEmcVsSjUrtEw, mOKi4ysh1CK2rLu8pKwBAQ...",12,4.06,5,2,1,0,0,2,2,7,7,4,1
4,394483,B1SdzXmYZvK7QZwZ1g_XCg,Marlene,8,2015-06-20 00:57:29,3,1,0,NaN,"hFQ9pR73raKHR031Y_skYw, DV1w2IaGhifIjdKLQReWXw...",0,3.78,0,0,0,0,0,0,0,0,0,0,0


In [17]:
us.columns

Index(['Unnamed: 0', 'user_id', 'name', 'review_count', 'yelping_since',
       'useful', 'funny', 'cool', 'elite', 'friends', 'fans', 'average_stars',
       'compliment_hot', 'compliment_more', 'compliment_profile',
       'compliment_cute', 'compliment_list', 'compliment_note',
       'compliment_plain', 'compliment_cool', 'compliment_funny',
       'compliment_writer', 'compliment_photos'],
      dtype='object')

In [ ]:
us.drop(['Unnamed: 0',''])

In [16]:
pd.DataFrame({'count':us.count(),'unique':us.nunique(),'null count':us.isnull().sum()})

,count,unique,null count
Unnamed: 0,81857,81857,0
user_id,81857,81857,0
name,81857,14886,0
review_count,81857,737,0
yelping_since,81857,81839,0
useful,81857,1221,0
funny,81857,796,0
cool,81857,889,0
elite,3584,197,78273
friends,81857,47521,0


In [21]:
# get the average of the # of compliments in each category
# could be that they are inclined to go to more restaurants due to more compliments
us.filter(regex='compliment').agg({'mean'})

,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
mean,2.228435,0.307561,0.182855,0.163651,0.059714,1.343172,2.749075,2.938796,2.938796,1.073066,1.067545


# Reviews

In [6]:
rv = pd.read_csv('sample_dataset/review_sample.csv')

In [7]:
rv.head()

,Unnamed: 0,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,4634629,xw37uw9b2wFwVRLXSCtgNg,tIau2r-ox1LhC38wXX6oew,IUoj0ORP1VAV9xx4dmHP9g,5.0,0,0,0,I love this location and have been a customer ...,2018-09-07 20:41:56
1,4349124,jKxK47dz6D-ZpKqmd-oTeA,q0lV9MoRYDhyoLAQ9X91iA,hihud--QRriCYZw1zZvW4g,1.0,0,0,0,I think that the food is not at all authentic ...,2018-01-03 03:57:16
2,6056428,boaG5qtsdwR3KBwQbvnfAg,kUE8PMxx2_HTT32T16Z8Aw,zT3IxSfolhu4qU0tV0pTjg,2.0,1,0,0,If you schedule your appointment late in the a...,2018-06-27 01:40:52
3,1713906,ah5BuOXlUwUIfSYHPa9Syg,0wOrT_OF0QN7bWlbgCc2ww,em__JLa6eEyYXzMbbVEahQ,5.0,0,0,0,Wow! What can I say about Shauna? Plenty of th...,2013-07-08 06:53:20
4,3794033,TB2cTa0ji3JaYbInQN_nqQ,oKhpMYvIntENZ5UzuoZWUg,1KhBzrqmU164bss64jR5uA,4.0,0,0,0,Fun night of karaoke. Food good...lady barten...,2016-03-16 05:56:01


In [ ]:
pd.DataFrame({'count':rv.count(),'unique':rv.nunique(),'null count':rv.isnull().sum()})